In [2]:
import time
import os, argparse
import numpy as np
from typing import List
import cv2
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from openvino.inference_engine import IECore
from IPython.display import HTML, FileLink, display

abs = os.getcwd() + "/"

ModuleNotFoundError: No module named 'openvino'

In [ ]:
onnx_path = "trained_models/onnx/its_train_ffa_3_9.pth"
ir_path = "trained_models/ir"
output_dir = abs + 'test_output/'
img_dir = abs + 'test/'
print("pred_dir:", output_dir)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
# Load IR network to Inference Engine
ie = IECore()
net_ir = ie.read_network(model=ir_path)
exec_net_ir = ie.load_network(network=net_ir, device_name="CPU")

# Get names of input and output layers
input_layer_ir = next(iter(exec_net_ir.input_info))
output_layer_ir = next(iter(exec_net_ir.outputs))

# Run the Inference on the Input image...
for img in os.listdir(img_dir):
    start_time = time.time()
    img = cv2.cvtColor(
    cv2.imread(str(img_dir + img)),
    cv2.COLOR_BGR2RGB,
    )
    res_ir = exec_net_ir.infer(inputs={input_layer_ir: img})
    res_ir = res_ir[output_layer_ir]
    end_time = time.perf_counter()
    print(
        f"Inference finished. Inference time: {end_time-start_time:.3f} seconds, "
        f"FPS: {1/(end_time-start_time):.2f}."
    )
    result_img = np.rint(
        cv2.resize(np.squeeze(res_ir), (img.shape[1], img.shape[0]))
    ).astype(np.uint8)
    # Save the output image
    cv2.imwrite(str(output_dir+img), cv2.cvtColor(result_img, cv2.COLOR_RGB2BGR))